<a href="https://colab.research.google.com/github/Tanushreedew/Web-Dev-Code/blob/main/lip_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from typing import List
from matplotlib import pyplot as plt
import imageio
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import os
import cv2
import dlib
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass


In [ ]:
detector = dlib.get_frontal_face_detector()

predictor = dlib.shape_predictor('/content/drive/MyDrive/tanu_data/shape_predictor_68_face_landmarks.dat')

def extract_frames_and_convert_to_tensor(eo_path):
    frames = []

    # Open the video file
    cap = cv2.VideoCapture(str(eo_path),cv2.CAP_ANY)

    # Read and process each frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break
      # hellop pranit yaan in the
        # Convert frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        expansion_factor = 1.3
        image = frame_gray
        faces = detector(frame_gray)


        for face_index, face in enumerate(faces):
            landmarks = predictor(image, face)

            mouth_points = []
            for n in range(48, 68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                mouth_points.append((x, y))


            x_min = min([point[0] for point in mouth_points])
            x_max = max([point[0] for point in mouth_points])
            y_min = min([point[1] for point in mouth_points])
            y_max = max([point[1] for point in mouth_points])

            width = x_max - x_min
            height = y_max - y_min
            expansion_width = int(width * expansion_factor)
            expansion_height = int(height * expansion_factor)
            x_min_expanded = max(0, x_min - int((expansion_width - width) / 2))
            y_min_expanded = max(0, y_min - int((expansion_height - height) / 2))
            x_max_expanded = min(image.shape[1], x_max + int((expansion_width - width) / 2))
            y_max_expanded = min(image.shape[0], y_max + int((expansion_height - height) / 2))


            cropped_mouth = image[y_min_expanded:y_max_expanded, x_min_expanded:x_max_expanded]





            # Resize the frame to a consistent shape
            resized_frame = cv2.resize(cropped_mouth, (140,46))
            expanded_frame = np.expand_dims(resized_frame, axis=-1)
            # reshaped_frame = np.expand_dims(frames, axis=-1)  # Add dimension for grayscale



            # Convert the frame tensor to images (NumPy arrays)
            # frame_images = tf.image.convert_image_dtype(resized_frame, tf.uint8).numpy()
            frames.append(expanded_frame)

    # Release the video capture
    cap.release()

    # Convert frames list to a TensorFlow tensor
    tensor_frames = tf.convert_to_tensor(frames, dtype=tf.float32)
    tensor_frames = tf.cast(frames, tf.float32)
    # frame_images = tf.image.convert_image_dtype(frames, tf.uint8).numpy()
    # mean =[]

    return tensor_frames


# Example usage
video_path = "/content/drive/MyDrive/test_data/test_video/bbwe2s.mpg"
tensor_frames = extract_frames_and_convert_to_tensor(video_path)
# tensor_frames = np.expand_dims(tensor_frames, axis=-1)
print(f"Tensor Frames: {tensor_frames.shape}")

Tensor Frames: (75, 46, 140, 1)


In [ ]:
# def load_video(path:str) -> List[float]:

#     cap = cv2.VideoCapture(path)
#     frames = []
#     for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
#         ret, frame = cap.read()
#         frame = tf.image.rgb_to_grayscale(frame)
#         frames.append(frame[190:236,80:220,:])
#     cap.release()

#     mean = tf.math.reduce_mean(frames)
#     std = tf.math.reduce_std(tf.cast(frames, tf.float32))
#     return tf.cast((frames - mean), tf.float32) / std

In [ ]:
vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]

encoder = LabelEncoder()
encoder.fit(vocab)
encoded_strings = encoder.transform(vocab)

num_unique_strings = len(encoder.classes_)
print(num_unique_strings)

encoded_tensor = tf.constant(encoded_strings, dtype=tf.int32)
print(encoded_tensor)

39
tf.Tensor(
[13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36
 37 38  2 12  1  3  4  5  6  7  8  9 10 11  0], shape=(39,), dtype=int32)


In [ ]:
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

The vocabulary is: ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' '] (size =40)


In [ ]:
def load_alignments(path:str) -> List[str]:
    with open(path, 'r') as f:
        lines = f.readlines()
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil':
            tokens = [*tokens,' ',line[2]]
    return char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

In [ ]:
# def encoded_string(string):
#     target_encoding = encoder.transform([string])[0]
#     target_encoding = tf.constant(target_encoding, dtype=tf.int32)
#     return target_encoding

# def process_video_alignment(video_path):
#     # Read the align file
#     align_path =video_path

#     with open(align_path, "r") as file:
#         lines = file.readlines()

#     # Extract the words and ignore time durations and "sil"
#     transcription = []
#     for line in lines[1:-1]:
#         tokens = line.split()
#         if tokens[2] != "sil":
#             transcription.append(tokens[2] + " ")

#     # Join the tokens into a single string
#     transcription_str = "".join(transcription).strip()
#     tensor_transcription = tf.stack(
#     [encoded_string(char) for char in transcription_str], axis=0
#     )

#     # Return the processed transcription string
#     return tensor_transcription

In [ ]:
# def decode_tensor(tensor, encoder):
#     decoded_chars = []
#     for encoded_value in tensor:
#         # encoded_value = encoded_value.numpy().astype(np.int64)
#         decoded_chars.append(encoder.inverse_transform([encoded_value])[0])
#     decoded_string = "".join(decoded_chars)
#     return decoded_string

In [ ]:
tf.config.run_functions_eagerly(True)
def together(path: str):

  path = bytes.decode(path.numpy())
  split_value = []
  tmp = ''
  for c in path:
    if c == '/':
        split_value.append(tmp)
        tmp = ''
    else:
        tmp += c
  if tmp:
    split_value.append(tmp)



  split_value_2 = []
  tmp = ''
  for c in split_value[-1]:
    if c == '.':
        split_value_2.append(tmp)
        tmp = ''
    else:
        tmp += c
  if tmp:
    split_value_2.append(tmp)

  file_name = split_value_2[0]

  # print(file_name)
  alignment_path = os.path.join('/content/drive/MyDrive/test_data/','test_alignments',f'{file_name}.align')
  alignment = load_alignments(alignment_path)

  frames = extract_frames_and_convert_to_tensor(path)
  return frames , alignment

In [ ]:
# def load_data(path: str):
#     path = bytes.decode(path.numpy())
#     file_name = path.split('/')[-1].split('.')[0]
#     # File name splitting for windows
#     #file_name = path.split('\\')[-1].split('.')[0]
#     video_path = os.path.join('/content/drive/MyDrive/test_data','test_video',f'{file_name}.mpg')
#     alignment_path = os.path.join('/content/drive/MyDrive/test_data','test_alignments',f'{file_name}.align')
#     frames = extract_frames_and_convert_to_tensor(video_path)
#     alignments = process_video_alignment(alignment_path)

#     return frames, alignments

In [ ]:
def mappable_function(path:str) ->List[str]:
    result = tf.py_function(together, [path], (tf.float32, tf.int64))
    return result

In [ ]:
tf.data.experimental.enable_debug_mode()
data = tf.data.Dataset.list_files('/content/drive/MyDrive/test_data/test_video/*.mpg')
data = data.map(mappable_function)
data = data.shuffle(1000, reshuffle_each_iteration=False)
# Determine the maximum shape for padding
max_frame_shape = tf.TensorShape([75, None, None,None])
max_alignment_shape = tf.TensorShape([40])

# Pad the dataset based on the maximum shape
data = data.padded_batch(2, padded_shapes=(max_frame_shape, max_alignment_shape))

data = data.prefetch(tf.data.AUTOTUNE)

train_ds = data.take(50)
val_ds = data.skip(50)

In [ ]:
len(data)

55

In [ ]:
len(train_ds)

50

In [ ]:
len(val_ds)

5

In [ ]:
frames, alignments = data.as_numpy_iterator().next()

In [ ]:
data.as_numpy_iterator().next()[0][0].shape

(75, 46, 140, 1)

In [ ]:
frames_train, alignments_train = train_ds.as_numpy_iterator().next()

In [ ]:
frames_val, alignments_val = val_ds.as_numpy_iterator().next()

In [ ]:
model = Sequential()
model.add(Conv3D(128, 3, input_shape=(75,46,140,1), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(256, 3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(75, 3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(TimeDistributed(Flatten()))

model.add(Bidirectional(LSTM(128, kernel_initializer='Orthogonal', return_sequences=True)))
model.add(Dropout(.5))

model.add(Bidirectional(LSTM(128, kernel_initializer='Orthogonal', return_sequences=True)))
model.add(Dropout(.5))

model.add(Dense(41, kernel_initializer='he_normal', activation='softmax'))




In [ ]:
model.input_shape

(None, 75, 46, 140, 1)

In [ ]:
model.output_shape

(None, 75, 41)

In [ ]:
sample = data.as_numpy_iterator()

In [ ]:
val = sample.next();

In [ ]:
yhat = model.predict(val[0])

1/1 [==============================] - 10s 10s/step


In [ ]:
def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [ ]:
def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss


In [ ]:
class ProduceExample(tf.keras.callbacks.Callback):
    def __init__(self, dataset) -> None:
        self.dataset = dataset.as_numpy_iterator()

    def on_epoch_end(self, epoch, logs=None) -> None:
        data = self.dataset.next()
        yhat = self.model.predict(data[0])
        decoded = tf.keras.backend.ctc_decode(yhat, [75,75], greedy=False)[0][0].numpy()
        for x in range(len(yhat)):
            print('Original:', tf.strings.reduce_join(num_to_char(data[1][x])).numpy().decode('utf-8'))
            print('Prediction:', tf.strings.reduce_join(num_to_char(decoded[x])).numpy().decode('utf-8'))
            print('~'*100)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss=CTCLoss)

In [ ]:
checkpoint_callback = ModelCheckpoint(os.path.join('models','checkpoint'), monitor='loss', save_weights_only=True)

In [ ]:
schedule_callback = LearningRateScheduler(scheduler)

In [ ]:
example_callback = ProduceExample(val_ds)

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[checkpoint_callback, schedule_callback, example_callback])

Epoch 1/100
1/1 [==============================] - 0s 68ms/step
Original: place blue with u four again
Prediction: 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: lay blue with r four please
Prediction: 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
50/50 [==============================] - 867s 12s/step - loss: 130.2278 - val_loss: 114.1153 - lr: 1.0000e-04
Epoch 2/100
1/1 [==============================] - 0s 73ms/step
Original: set blue at l four soon
Prediction:  
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: lay blue in d five again
Prediction:  
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
50/50 [==============================] - 610s 7s/step - loss: 109.2243 - val_loss: 99.9697 - lr: 1.0000e-04
Epoch 3/100
1/1 [==============================] -

StopIteration: ignored

In [ ]:
# np.unique(alignments)

array([ 0,  1, 14, 15, 18, 19, 20, 22, 25, 27, 28, 29, 31, 32, 33, 34, 35,
       38, 39], dtype=int32)

In [ ]:
# data.classes_ = np.unique(alignments)

In [ ]:
# print(data.classes_)

[ 0  1 14 15 18 19 20 22 25 27 28 29 31 32 33 34 35 38 39]


In [ ]:
# model.save_weights('/content/drive/MyDrive/test_data/model_weights.h5')
# import tensorflow as tf
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.losses import CategoricalCrossentropy

# # Define the loss function
# loss_function = CategoricalCrossentropy()

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=0.001), loss=loss_function, metrics=['accuracy'])

# previous_weights_path = '/content/drive/MyDrive/test_data/model_weights.h5'
# model.load_weights(previous_weights_path)
# # Define the training data and labels
# train_data = frames_train
# train_labels = alignments_train

# # Define the validation data and labels
# val_data = frames_val
# val_labels = alignments_val

# # Convert labels to one-hot encoding if needed
# train_labels_one_hot = tf.one_hot(train_labels, depth=40)
# val_labels_one_hot = tf.one_hot(val_labels, depth=40)

# class PrintAccuracyCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs=None):
#         print(f"Accuracy: {logs['accuracy']}")

# # Train the model with validation data
# model.fit(train_data, train_labels_one_hot, batch_size=32, epochs=10, validation_data=(val_data, val_labels_one_hot))



Epoch 1/10


ValueError: ignored